In [1]:
!pip install pandas

!pip install chromedriver-autoinstaller

!pip install BeautifulSoup4
!pip install lxml
!pip install selenium
!pip install selenium --upgrade
!pip install webdriver-manager

In [2]:
import os
import sys
from datetime import datetime
import time
import requests
import re

import pandas as pd

import chromedriver_autoinstaller
import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [53]:
# headless mode

option = webdriver.ChromeOptions()
option.add_argument("--headless")

service = Service(executable_path='chromedriver.exe')
browser = webdriver.Chrome(service=service, options=option)

html = browser.page_source
soup = BeautifulSoup(html, 'lxml')

url = r"https://www.catch.co.kr/"

browser.maximize_window()
print(browser.get_window_size())

browser.get(url)


{'width': 800, 'height': 600}


In [54]:
from accounts import ct

ID = ct.ID
PW = ct.PW

In [55]:
# 로그인

browser.find_element(By.CSS_SELECTOR, "a.login").click()

browser.find_element(By.CSS_SELECTOR, "input#id_login").send_keys(ID)
time.sleep(1)
browser.find_element(By.CSS_SELECTOR, "input#pw_login").send_keys(PW)

browser.find_element(By.CSS_SELECTOR, "input#pw_login").send_keys(Keys.ENTER)

In [56]:
# 기업 검색 (input)
query = browser.find_element(By.CSS_SELECTOR, "input")

query.send_keys(input("리뷰 확인하기: "))
query.send_keys(Keys.ENTER)

In [57]:
### 팝업 뜨면 끄기 (예외 처리)

try : 
    browser.find_element(By.CSS_SELECTOR, 'button.today > span').click()
    time.sleep(2)

except : 
    pass

In [58]:
# 검색한 기업(최상위 노출) 클릭 
browser.find_element(By.CSS_SELECTOR, "p.name > a").click()
time.sleep(2)

In [59]:
# '현직자 리뷰' 탭 클릭
browser.find_element(By.XPATH, r'//*[@id="Contents"]/div[1]/div[1]/div[2]/ul/li[4]/a').click()
time.sleep(1)

In [60]:
# 핵심 리뷰만 조회 - 정규직/현직자가 3년 이내 작성한 리뷰
browser.find_element(By.CSS_SELECTOR, 'div.pd_type2 > div > a').click()
time.sleep(1)

In [61]:
# 긍정 리뷰만 조회
good_rv = browser.find_element(By.CSS_SELECTOR, "li:nth-child(1) > p:nth-child(4)").text
good_rv = good_rv.replace(good_rv[:5], "").replace('\n', ' ') 
good_rv

'눈치 볼 필요없이 출퇴근 가능, 지하철이나 교통편이 2. 5 호선을 갈아타기 때문에 서울 전역에서 출근 가능, 휴식실 구비 생각보다 넓은 공간과 쾌적함.'

In [62]:
# 부정 리뷰만 조회
bad_rv = browser.find_element(By.CSS_SELECTOR, "li:nth-child(1) > p:nth-child(5)").text
bad_rv = bad_rv.replace(bad_rv[:4], "").replace('\n', ' ') 
bad_rv

'높은 업무 요구와 가끔 프로젝트를 혼자 혹은 둘이서 맡을 때가 있음 타지역 사람들은 출퇴근이 많이 힘들다는 것, (서울 외곽으로 집입하기 힘들기 때문)'

In [63]:
good_df = pd.DataFrame({"긍정적 평가" : [good_rv]})
good_df


,긍정적 평가
0,"눈치 볼 필요없이 출퇴근 가능, 지하철이나 교통편이 2. 5 호선을 갈아타기 때문에..."


In [64]:
bad_df = pd.DataFrame({"부정적 평가" : [bad_rv]})
bad_df

,부정적 평가
0,높은 업무 요구와 가끔 프로젝트를 혼자 혹은 둘이서 맡을 때가 있음 타지역 사람들은...


In [65]:
good_df.join(bad_df)

,긍정적 평가,부정적 평가
0,"눈치 볼 필요없이 출퇴근 가능, 지하철이나 교통편이 2. 5 호선을 갈아타기 때문에...",높은 업무 요구와 가끔 프로젝트를 혼자 혹은 둘이서 맡을 때가 있음 타지역 사람들은...


In [70]:
# Web Crawling

n_btn = browser.find_element(By.CSS_SELECTOR, 'a.ico.next')


good_lst = []
bad_lst = []


i = 0 

pre_page = 0

while True : 

    time.sleep(1)

    # 현재 페이지의 숫자
    crt_btn = browser.find_element(By.CSS_SELECTOR, 'div.pd_type2 > p > a.selected')
    crt_page = int(crt_btn.text)

    # 다음 버튼 눌러도 같은 페이지의 경우 (마지막 페이지)
    if crt_page == pre_page:
        break


    for p in range(1, 5) : 

        grv = browser.find_element(By.CSS_SELECTOR, f"li:nth-child({p}) > p:nth-child(4)").text
        brv = browser.find_element(By.CSS_SELECTOR, f"li:nth-child({p}) > p:nth-child(5)").text

        grv_clean = grv.replace(grv[:5], "").replace('\n', ' ') 
        brv_clean = brv.replace(brv[:4], "").replace('\n', ' ') 

        good_lst.append(grv_clean)
        bad_lst.append(brv_clean)


    time.sleep(2)

    # WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, f"li:nth-child({i}) > p:nth-child(4)")))


# 다음 페이지로 이동하기 위한 버튼을 찾기.
    try:
        # 이전페이지의 번호를 갱신
        pre_page = crt_page

        # 다음 버튼 클릭
        n_btn.click()

        # 버튼 누르고나서 정보를 가져올때까지 슬립(몇초동안)걸어두기.
        time.sleep(1)

    except:
        break

browser.close()

good_lst = list(set(good_lst))
bad_lst = list(set(bad_lst))


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"li:nth-child(3) > p:nth-child(4)"}
  (Session info: headless chrome=114.0.5735.199); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00DDA813+48355]
	(No symbol) [0x00D6C4B1]
	(No symbol) [0x00C75358]
	(No symbol) [0x00CA09A5]
	(No symbol) [0x00CA0B3B]
	(No symbol) [0x00CCE232]
	(No symbol) [0x00CBA784]
	(No symbol) [0x00CCC922]
	(No symbol) [0x00CBA536]
	(No symbol) [0x00C982DC]
	(No symbol) [0x00C993DD]
	GetHandleVerifier [0x0103AABD+2539405]
	GetHandleVerifier [0x0107A78F+2800735]
	GetHandleVerifier [0x0107456C+2775612]
	GetHandleVerifier [0x00E651E0+616112]
	(No symbol) [0x00D75F8C]
	(No symbol) [0x00D72328]
	(No symbol) [0x00D7240B]
	(No symbol) [0x00D64FF7]
	BaseThreadInitThunk [0x759F7D59+25]
	RtlInitializeExceptionChain [0x76F2

In [72]:
browser.close()

In [71]:
len(good_lst), len(bad_lst)

(2, 2)

In [46]:
good_lst

['성과가 좋다면 고속 승진 가능 및 다양한 업무와 TF활동 경험 가능',
 '사무실 환경이 쾌적하고, 재택 근무 병행으로 주에 1회 정도만 출근(물론 팀바팀)',
 '일한 만큼은 주는것 같다, 쿠팡이라는 네임드, 쏘씨오 패스같은 독함이 있다면 무한승진 가능',
 '네임벨류좋고 아직까지 재택기조 유지(부서별 다름) 일을 배우기 좋다',
 '연차는 지유롭게 사용가능, 본인 업무만 책임지면 아무도 뭐라안함, 회사 네임벨류, 연봉기준 산정의 성과금, 유연근무제, 사람들이 좋은편, 제조나 건설보단 수평적이긴함, 성과에 따라 빠른 진급도 가능',
 '눈치안봄 배송하면서 혼자일해서 자유로움 연차마음대로사용가능 입사하면 연차 선지급',
 '타회사처럼 상사개념이아닌 님문화라 수평적인관계가좋다 휴가 퇴근 눈치안보는것도',
 '내가 하는 일에 숙련만되면 그 누구도 터치가 없는 일. 명절 쿠팡 10만 보너스같은 캐시지급',
 '부바부가 심하지만 자유롭고 팀 내 분위기는 좋은 편이라 나만 잘 하면 되는 분위기를 만들어 준다. 회사 자체의 플로우가 빠르기 때문에 진취적이고, 일하는데 보람을 느끼는 사람들이 일하기 좋은 환경이다. 수동적이고 변화를 싫어하고 안정을 추구하는 업무스타일은 힘들어할 것이다. 첫 입사시 연봉협상을 잘 하면 버는만큼 일한다고 느껴질것이다.',
 '계약된 근로시간 외 잔업이 없으며 야근을 강요하지 않는다. 월급을 위해 추가 근무 요청을 할 수 있다. 4시간만 근무하는 분들도 계시다. 한달 휴일을 미리 지정해서 쓸 수 있다. 재택 근무 가능하다. 회식을 강요받은 적 없다. 팀장과 잘 맞으면 장기 근무로 좋다.',
 '퇴근이 자유롭고 자율근무 복장에 재택근무가 활성화되어 있습니다',
 '재택근무가 최고의 복지. 같이 일하는 사람들도 모나지 않고 전부 능력있어서 배울 점 많음.',
 '눈치 볼 필요없이 칼퇴 및 연차 사용이 가능합니다. 사무실 내 자판기 가격300원에 사용가능하며 지하철역과 버스정류장 바로 앞이라 교통편이 좋아요! 워라밸 보장됩니다!',
 '칼퇴 보장

In [47]:
good_lst[-5:], bad_lst[-5:]

(['사원들에게 기회를 많이 줍니다. 수평적인 문화와 여러 보수적인 기업문화가 없어서 좋습니다.',
  '수평적인 관계와 자유로운 부서 이동, 칼퇴 보장되는 점, 연차 자유',
  '상사가 실적을 쪼지 않으면 좋음. 그리고 법을 잘 키는 점은 좋은 편이다',
  '들어오면 이력서 빵빵하게 채워서 나갈 수 있음, 커리어 개발 면에 있어서는 추천',
  '성장하는 회사라는것을 몸소 느낄 수 있다. 커리어에 도움이 된다.'],
 ['업무강도가 좀 높은 편입니다. 개인의 능력을 중요시하다보니, 공동체의식이 좀 부족합니다',
  '자유로운 부서 이동, 야근수당 없는 점, 구내식당 도시락 별로',
  '직원 관리는 전혀 안되고 실적으로만 쪼니 사람들이 떠나는 이유를 알겠는 회사. 직원들을 생각한다면 실적으로 쪼기전에 직원들 심리 검사를 하길',
  '프로세스가 없고 프로세스를 만들어 놓아도 지켜지지 않음',
  '성장하는 속도에 비해 체계가 잡히지 않는다. 이 부분은 아쉽다.'])

In [48]:
good_df = pd.DataFrame({"긍정적 평가" : good_lst})
bad_df = pd.DataFrame({"부정적 평가" : bad_lst})

good_df.shape, bad_df.shape

((34, 1), (34, 1))

In [49]:
good_df.tail()

,긍정적 평가
29,사원들에게 기회를 많이 줍니다. 수평적인 문화와 여러 보수적인 기업문화가 없어서 좋...
30,"수평적인 관계와 자유로운 부서 이동, 칼퇴 보장되는 점, 연차 자유"
31,상사가 실적을 쪼지 않으면 좋음. 그리고 법을 잘 키는 점은 좋은 편이다
32,"들어오면 이력서 빵빵하게 채워서 나갈 수 있음, 커리어 개발 면에 있어서는 추천"
33,성장하는 회사라는것을 몸소 느낄 수 있다. 커리어에 도움이 된다.


In [50]:
bad_df.tail()

,부정적 평가
29,"업무강도가 좀 높은 편입니다. 개인의 능력을 중요시하다보니, 공동체의식이 좀 부족합니다"
30,"자유로운 부서 이동, 야근수당 없는 점, 구내식당 도시락 별로"
31,직원 관리는 전혀 안되고 실적으로만 쪼니 사람들이 떠나는 이유를 알겠는 회사. 직원...
32,프로세스가 없고 프로세스를 만들어 놓아도 지켜지지 않음
33,성장하는 속도에 비해 체계가 잡히지 않는다. 이 부분은 아쉽다.


In [51]:
pros_cons = good_df.join(bad_df)
pros_cons

,긍정적 평가,부정적 평가
0,성과가 좋다면 고속 승진 가능 및 다양한 업무와 TF활동 경험 가능,"워라밸X 군대 수직적 조직문화, 초과 근무수당 x 바쁜 시기는 많이 바쁘다"
1,"사무실 환경이 쾌적하고, 재택 근무 병행으로 주에 1회 정도만 출근(물론 팀바팀)","포괄연봉제로 별도 야근 수당 없음, 경력직 온보딩 체계적이지 않아 초기 적응 어려울..."
2,"일한 만큼은 주는것 같다, 쿠팡이라는 네임드, 쏘씨오 패스같은 독함이 있다면 무한승...","기준없는 업무 정책변경으로 이해상충 업무 많음, 계속 몬가를 하지만 성과는 그들만의..."
3,네임벨류좋고 아직까지 재택기조 유지(부서별 다름) 일을 배우기 좋다,너무 빠르게 돌아가기 때문에 속도를 낼 수 없는 사람들은 적응하기 힘들다
4,"연차는 지유롭게 사용가능, 본인 업무만 책임지면 아무도 뭐라안함, 회사 네임벨류, ...","성과를 매우 중시, 성과를 운에 맡겨야 할 때가 있음, 상식에서 벗어난 행동을 하는..."
5,눈치안봄 배송하면서 혼자일해서 자유로움 연차마음대로사용가능 입사하면 연차 선지급,몸이 많이 힘들고 가끔씩 관리자들이 이해가 안되는 면이 있음.
6,타회사처럼 상사개념이아닌 님문화라 수평적인관계가좋다 휴가 퇴근 눈치안보는것도,업무량이많은거에비해 경영진의 직원복지대처가아쉽다 너무많은 물량에 힘들다
7,내가 하는 일에 숙련만되면 그 누구도 터치가 없는 일. 명절 쿠팡 10만 보너스같은...,빨리끝나도 협업으로인한 늦은퇴근 정해져있는퇴근시간 그이후에만가능
8,부바부가 심하지만 자유롭고 팀 내 분위기는 좋은 편이라 나만 잘 하면 되는 분위기를...,업무중심적이다보니 팀 성과에 따라 분위기는 그때그때 다를 수 있다. 윗선 눈치를 많...
9,계약된 근로시간 외 잔업이 없으며 야근을 강요하지 않는다. 월급을 위해 추가 근무 ...,"정해진 휴일을 조정해달라는 요청이 간혹 있는데, 팀장이 어떤 사람인가에 따라 케바케..."


In [52]:
# 2023-07-04 21:00 commit
# 2023-07-05 13:05 modified

## to .py file

In [ ]:
%%writefile catch.py

# pip install

!pip install pandas

!pip install chromedriver-autoinstaller

!pip install BeautifulSoup4
!pip install lxml
!pip install selenium
!pip install selenium --upgrade
!pip install webdriver-manager



# import

import os
import sys
from datetime import datetime
import time
import requests
import re

import pandas as pd

import chromedriver_autoinstaller
import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



# ChromeBrowser - headless mode

option = webdriver.ChromeOptions()
option.add_argument#("--headless")

service = Service(executable_path='chromedriver.exe')
browser = webdriver.Chrome(service=service, options=option)

html = browser.page_source
soup = BeautifulSoup(html, 'lxml')

url = r"https://www.catch.co.kr/"

browser.maximize_window()
print(browser.get_window_size())

browser.get(url)



# 로그인

from accounts import ct

ID = ct.ID
PW = ct.PW


browser.find_element(By.CSS_SELECTOR, "a.login").click()

browser.find_element(By.CSS_SELECTOR, "input#id_login").send_keys(ID)
time.sleep(1)
browser.find_element(By.CSS_SELECTOR, "input#pw_login").send_keys(PW)

browser.find_element(By.CSS_SELECTOR, "input#pw_login").send_keys(Keys.ENTER)



# 기업 검색 (input)
query = browser.find_element(By.CSS_SELECTOR, "input")

query.send_keys(input("리뷰 확인하기: "))
query.send_keys(Keys.ENTER)

try : 
    browser.find_element(By.CSS_SELECTOR, 'button.today > span').click()
    time.sleep(2)

except : 
    pass


browser.find_element(By.CSS_SELECTOR, "p.name > a").click()
time.sleep(2)

browser.find_element(By.XPATH, r'//*[@id="Contents"]/div[1]/div[1]/div[2]/ul/li[4]/a').click()
time.sleep(1)

browser.find_element(By.CSS_SELECTOR, 'div.pd_type2 > div > a').click()
time.sleep(1)



# reviews Crawling


n_btn = browser.find_element(By.CSS_SELECTOR, 'a.ico.next')


good_lst = []
bad_lst = []


i = 0 

pre_page = 0

while True : 

    # 현재 페이지의 숫자
    crt_btn = browser.find_element(By.CSS_SELECTOR, 'div.pd_type2 > p > a.selected')
    crt_page = int(crt_btn.text)

    # 다음 버튼 눌러도 같은 페이지의 경우 (마지막 페이지)
    if crt_page == pre_page:
        break


    for p in range(1, 5) : 

        grv = browser.find_element(By.CSS_SELECTOR, f"li:nth-child({p}) > p:nth-child(4)").text
        brv = browser.find_element(By.CSS_SELECTOR, f"li:nth-child({p}) > p:nth-child(5)").text

        grv_clean = grv.replace(grv[:5], "").replace('\n', ' ') 
        brv_clean = brv.replace(brv[:4], "").replace('\n', ' ') 

        good_lst.append(grv_clean)
        bad_lst.append(brv_clean)


    time.sleep(2)

    # WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, f"li:nth-child({i}) > p:nth-child(4)")))


# 다음 페이지로 이동하기 위한 버튼을 찾기.
    try:
        # 이전페이지의 번호를 갱신
        pre_page = crt_page

        # 다음 버튼 클릭
        n_btn.click()

        # 버튼 누르고나서 정보를 가져올때까지 슬립(몇초동안)걸어두기.
        time.sleep(1)

    except:
        break


good_lst = list(set(good_lst))
bad_lst = list(set(bad_lst))



# List 2 DataFrame

good_df = pd.DataFrame({"긍정적 평가" : good_lst})
bad_df = pd.DataFrame({"부정적 평가" : bad_lst})

pros_cons = good_df.join(bad_df)        # 긍정 부정 병합
